<a href="https://colab.research.google.com/github/lxdragoon/Modified-mnist/blob/master/Litao_Li_Queen's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
!wget https://www.dropbox.com/sh/jn8p1pvpgjy3b9b/AABCIQRx7cVR1Vn0Cwqqqxjya/comp551w18-modified-mnist.zip
!unzip comp551w18-modified-mnist.zip

In [0]:
x_ = np.loadtxt("train_x.csv", delimiter=",")# load from text
y_ = np.loadtxt("train_y.csv", delimiter=",")
x_train = x_.reshape(-1, 64, 64) # reshape
y_train = y_.reshape(-1, 1)

In [0]:
# normalize to 0-1, optional
x_train = x_train/255

In [0]:
# make y into one hot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(sparse = False)

In [0]:
# reshape to 1 dimension to feed into model
x_train = x_train.reshape(len(x_train),4096)


In [7]:
# transform labels into one hot encoding
y_train = onehot.fit_transform(y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
# split training data into train and test
import sklearn.model_selection as sk

X_train, X_test, Y_train, Y_test =  sk.train_test_split(x_train,y_train,test_size=0.2, random_state = 42)


## Logistic Regression

In [0]:
learning_rate = 0.01
training_epochs = 50
batch_size = 100
display_step = 10

x = tf.placeholder(tf.float32, [None, 4096])
y = tf.placeholder(tf.float32, [None, 10])

# model weights                                                   
W = tf.Variable(tf.zeros([4096, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables
init = tf.global_variables_initializer()

In [37]:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X_train)/batch_size)
        #temp = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
        #temp = temp.shuffle(1000).batch(batch_size)
        #iterator = temp.make_initializable_iterator()
        #next_element = iterator.get_next()
        #sess.run(iterator.initializer)
        
        # Loop over all batches
        n=0
        for i in range(total_batch):
            if n+batch_size >= len(X_train):
              n=0
            #batch_xs, batch_ys = sess.run(next_element)
            batch_xs = X_train[n:n+batch_size]
            batch_ys = Y_train[n:n+batch_size]
            n += batch_size
            
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            
            
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))

Epoch: 0010 cost= 2.261566154
Epoch: 0020 cost= 2.222527747
Epoch: 0030 cost= 2.192996626
Epoch: 0040 cost= 2.168911277
Epoch: 0050 cost= 2.148548626
Optimization Finished!
Accuracy: 0.1298


##CNN

In [0]:
# Training Parameters
learning_rate = 0.01
num_steps = 1000
batch_size = 128
display_step = 20

# Network Parameters
num_input = 4096
num_classes = 10 # total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [0]:
# wrappers
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 64, 64, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    
        # Convolution Layer
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv4, k=2)
    
        # Convolution Layer
    conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
    # Max Pooling (down-sampling)
    conv5 = maxpool2d(conv5, k=2)
    
        # Convolution Layer
    conv6 = conv2d(conv5, weights['wc6'], biases['bc6'])
    # Max Pooling (down-sampling)
    conv6 = maxpool2d(conv6, k=2)
    
        # Convolution Layer
    conv7 = conv2d(conv6, weights['wc7'], biases['bc7'])
    # Max Pooling (down-sampling)
    conv7 = maxpool2d(conv7, k=2)
    
        # Convolution Layer
    conv8 = conv2d(conv7, weights['wc8'], biases['bc8'])
    # Max Pooling (down-sampling)
    conv8 = maxpool2d(conv8, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    #fc1 = tf.contrib.layers.flatten(conv2)
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [0]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc4': tf.Variable(tf.random_normal([3, 3, 128, 128])),
    'wc5': tf.Variable(tf.random_normal([3, 3, 128, 128])),
    'wc6': tf.Variable(tf.random_normal([3, 3, 128, 128])),
    'wc7': tf.Variable(tf.random_normal([3, 3, 128, 128])),
    'wc8': tf.Variable(tf.random_normal([3, 3, 128, 128])),
    
    'wd1': tf.Variable(tf.random_normal([8*8*128, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bc4': tf.Variable(tf.random_normal([128])),
    'bc5': tf.Variable(tf.random_normal([128])),
    'bc6': tf.Variable(tf.random_normal([128])),
    'bc7': tf.Variable(tf.random_normal([128])),
    'bc8': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [44]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    n=0
    for step in range(1, num_steps+1):
        if n+batch_size >= 40000:
          n=0
        batch_x = X_train[n:n+batch_size]
        batch_y = Y_train[n:n+batch_size]
        n = n+batch_size
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 500 test data points
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test[500:1000],
                                      Y: Y_test[500:1000],
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 3190521.5000, Training Accuracy= 0.148
Step 20, Minibatch Loss= 88746.1172, Training Accuracy= 0.148
Step 40, Minibatch Loss= 23000.4199, Training Accuracy= 0.164
Step 60, Minibatch Loss= 14551.3955, Training Accuracy= 0.102
Step 80, Minibatch Loss= 8766.7188, Training Accuracy= 0.164
Step 100, Minibatch Loss= 5913.6172, Training Accuracy= 0.109
Step 120, Minibatch Loss= 3734.3452, Training Accuracy= 0.125
Step 140, Minibatch Loss= 2489.9158, Training Accuracy= 0.078
Step 160, Minibatch Loss= 1129.3057, Training Accuracy= 0.117
Step 180, Minibatch Loss= 509.2296, Training Accuracy= 0.141
Step 200, Minibatch Loss= 160.7523, Training Accuracy= 0.109
Step 220, Minibatch Loss= 44.1629, Training Accuracy= 0.086
Step 240, Minibatch Loss= 15.5151, Training Accuracy= 0.125
Step 260, Minibatch Loss= 14.2335, Training Accuracy= 0.078
Step 280, Minibatch Loss= 8.9820, Training Accuracy= 0.086
Step 300, Minibatch Loss= 15.9836, Training Accuracy= 0.102
Step 320, Minibatch L

## RNN

In [0]:

from tensorflow.contrib import rnn


In [0]:
tf.reset_default_graph()
# Training Parameters
learning_rate = 0.02
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 64 # data input: 64x64
timesteps = 64 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [0]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [0]:
#tf.reset_default_graph()
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [54]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    n = 0
    for step in range(1, training_steps+1):
        if n+batch_size >= 40000:
          n= 0 
        batch_x = X_train[n:n+batch_size]
        batch_y = Y_train[n:n+batch_size]
        n = n+batch_size
        # Reshape data to get 64 seq of 64 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 500 test data points
    test_data = X_test[1000:1500].reshape((-1, timesteps, num_input))
    test_label = Y_test[1000:1500]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 4.3304, Training Accuracy= 0.148
Step 200, Minibatch Loss= 2.2809, Training Accuracy= 0.125
Step 400, Minibatch Loss= 2.2552, Training Accuracy= 0.203
Step 600, Minibatch Loss= 2.2758, Training Accuracy= 0.102
Step 800, Minibatch Loss= 2.2975, Training Accuracy= 0.109
Step 1000, Minibatch Loss= 2.2795, Training Accuracy= 0.141
Step 1200, Minibatch Loss= 2.2633, Training Accuracy= 0.102
Step 1400, Minibatch Loss= 2.2359, Training Accuracy= 0.180
Step 1600, Minibatch Loss= 2.2883, Training Accuracy= 0.188
Step 1800, Minibatch Loss= 2.3011, Training Accuracy= 0.164
Step 2000, Minibatch Loss= 2.2954, Training Accuracy= 0.109
Step 2200, Minibatch Loss= 2.2864, Training Accuracy= 0.172
Step 2400, Minibatch Loss= 2.2724, Training Accuracy= 0.156
Step 2600, Minibatch Loss= 2.2821, Training Accuracy= 0.172
Step 2800, Minibatch Loss= 2.2882, Training Accuracy= 0.133
Step 3000, Minibatch Loss= 2.2554, Training Accuracy= 0.148
Step 3200, Minibatch Loss= 2.2895, Training Acc

# Gated CNN

In [0]:
# Training Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 100
display_step = 20

# Network Parameters
num_input = 4096
num_classes = 10 #total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [0]:
def conv2d(x, W, b, V, c, num, strides=1):
    # Conv2D wrapper, with bias and gates
    #paddings = tf.constant([[0,0],[0,0],[num,0],[0,0]])
    #x = tf.pad(x, paddings)
    A = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    B = tf.nn.conv2d(x, V, strides=[1, strides, strides, 1], padding='SAME')
    A = tf.nn.bias_add(A, b)
    B = tf.nn.bias_add(B, c)
    return tf.multiply(A, tf.nn.sigmoid(B))


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    
    x = tf.reshape(x, shape=[-1, 64, 64, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], weights['wv1'], biases['bv1'], weights['c1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], weights['wv2'], biases['bv2'], weights['c2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'], weights['wv3'], biases['bv3'], weights['c3'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv3, k=2)

    
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [0]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    
    'c1': 5,
    
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'c2': 3,
    
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wv3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'c3': 3,
 
    'wd1': tf.Variable(tf.random_normal([16*16*128, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bv1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bv2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bv3': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [58]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    n=0
    for step in range(1, num_steps+1):
        if n+batch_size >= 40000:
          n=0
        batch_x = X_train[n:n+batch_size]
        batch_y = Y_train[n:n+batch_size]
        n = n+batch_size
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 500 test data points
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test[2000:2500],
                                      Y: Y_test[2000:2500],
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 24301048.0000, Training Accuracy= 0.140
Step 20, Minibatch Loss= 1239636.0000, Training Accuracy= 0.110
Step 40, Minibatch Loss= 1312.2007, Training Accuracy= 0.100
Step 60, Minibatch Loss= 2.3287, Training Accuracy= 0.110
Step 80, Minibatch Loss= 2.3035, Training Accuracy= 0.080
Step 100, Minibatch Loss= 42.6643, Training Accuracy= 0.110
Step 120, Minibatch Loss= 2.3127, Training Accuracy= 0.070
Step 140, Minibatch Loss= 2.3184, Training Accuracy= 0.090
Step 160, Minibatch Loss= 145.9015, Training Accuracy= 0.100
Step 180, Minibatch Loss= 2.3093, Training Accuracy= 0.100
Step 200, Minibatch Loss= 2.2941, Training Accuracy= 0.120
Step 220, Minibatch Loss= 2.2717, Training Accuracy= 0.100
Step 240, Minibatch Loss= 2.2957, Training Accuracy= 0.160
Step 260, Minibatch Loss= 2.3218, Training Accuracy= 0.100
Step 280, Minibatch Loss= 2.2971, Training Accuracy= 0.130
Step 300, Minibatch Loss= 2.3041, Training Accuracy= 0.140
Step 320, Minibatch Loss= 2.3038, Training 